# Обработка

In [ ]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 47.1 MB/s eta 0:00:00


In [ ]:
from pymorphy3 import MorphAnalyzer
from random import sample, choice
morph = MorphAnalyzer()

In [ ]:
def conjugate(name, case):
  verb = ['VERB', 'INFN', 'PRTS', 'GRND']
  words = name.split()
  parsed = [morph.parse(word) for word in words]
  isverb = list(filter(lambda x: x[0].tag.POS in verb, parsed))
  if isverb:
    return words
  conjugated = []
  for i, pars in enumerate(parsed):
    if pars[0].tag.POS in ['NOUN', 'ADJF', 'PRTF', 'NUMR']:
      try:
        var = 0
        while pars[var].tag.case != 'nomn':
          var += 1
        conjugated.append(pars[var].inflect({case}).word.capitalize())
      except:
        conjugated.append(words[i])
    else:
      conjugated.append(words[i])
  return conjugated

In [ ]:
def change(sentence, eng, rus, case, type, places, new_rus, new_eng):
  start, finish = map(int, places.split(','))
  punctuation = sentence.split(rus)[1][0]
  conjugated = conjugate(new_rus, case)
  new_sentence = sentence.replace(eng, new_eng, 1).replace(rus, ' '.join(conjugated))
  lines = new_sentence.split('\n')
  intent = lines[3].split()[2]
  # lines.pop(0) # убираем id
  i = 0
  while lines[i].startswith('#') or int(lines[i].split('\t')[0]) < start:
    i += 1
  while i != len(lines) and int(lines[i].split('\t')[0]) <= finish:
    lines.pop(i)
  numeration = start
  for j, word in enumerate(conjugated):
    if j == 0:
      new_line = f'{numeration}\t{word}\t{intent}\tB-{type}'
    else:
      new_line = f'{numeration}\t{word}\t{intent}\tI-{type}'
    if j == len(conjugated) - 1 and punctuation in ['.', ',']:
      new_line = new_line.replace(word, word + punctuation)
    lines.insert(i, new_line)
    i += 1
    numeration += 1
  while i != len(lines):
    lines[i] = lines[i].replace(lines[i].split('\t')[0], str(numeration))
    i += 1
    numeration += 1
  return '\n'.join(lines)

In [ ]:
def generation(example, gap, filler, generated, size = 1, album = False):
  with open(example, 'r') as f:
    sentences = f.read().split('\n\n')
  with open(gap, 'r') as f:
    words = f.read().split('\n\n')
  with open(filler, 'r') as f:
    new_words = f.read().split('\n')
  paired = list(zip(sentences, words))
  for new_word in new_words:
    new = new_word.split('\t')
    for sentence, word in sample(paired, size):
      old = word.split('\n')
      try:
        if album:
          start1, finish1 = map(int, old[5].split(','))
          start2, finish2 = map(int, old[10].split(','))
          gap = len(new[0].split()) - (finish1 - start1) - 1
          start2 += gap
          finish2 += gap
          old[10] = f'{start2},{finish2}'
          changed = change(sentence, *old[1:6], *new[:2])
          changed = change(changed, *old[6:], *new[2:]) # changed = change('id\n'+changed, *old[6:], *new[2:])
        else:
          changed = change(sentence, *old[1:], *new)
      except Exception as e:
        print(e, old, new)
      with open(generated, 'a') as f:
        f.write(changed)
        f.write('\n\n')

In [ ]:
def changename(sentence, eng, rus, case, new_rus, new_eng):
  conjugated = conjugate(new_rus, case)
  new_sentence = sentence.replace(eng, new_eng).replace(rus, ' '.join(conjugated))
  lines = new_sentence.split('\n')
  # lines.pop(0) # убираем id
  return '\n'.join(lines)

In [ ]:
def generationnames(example, gap, filler, generated, size = 1):
  with open(example, 'r') as f:
    sentences = f.read().split('\n\n')
  with open(gap, 'r') as f:
    words = f.read().split('\n\n')
  with open(filler, 'r') as f:
    new_words = f.read().split('\n')
  paired = list(zip(sentences, words))
  for new_word in new_words:
    new = new_word.split('\t')
    for sentence, word in sample(paired, size):
      old = word.split('\n')
      if len(old) == 7:
        changed = changename(sentence, *old[1:4], *new)
        pair = choice(new_words)
        changed = changename(changed, *old[4:], *pair.split('\t')) # changed = changename('id\n'+changed, *old[4:], *pair.split('\t'))
      else:
        changed = changename(sentence, *old[1:], *new)
      with open(generated, 'a') as f:
        f.write(changed)
        f.write('\n\n')

In [ ]:
import os

files = os.listdir('/content/drive/MyDrive/examples')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
types = [f.split('.')[0] for f in files if f.endswith('conll')]

In [ ]:
for type in types:
  with open(f'/content/drive/MyDrive/gaps/{type}.conll', 'r') as f:
    names = f.read().split('\n')
    names = [n.strip() for n in names]
  with open(f'/content/drive/MyDrive/gaps/{type}.conll', 'w') as f:
    f.write('\n'.join(names))

In [ ]:
for type in types:
  if type != 'names':
      generation(f'/content/drive/MyDrive/examples/{type}.conll', f'/content/drive/MyDrive/gaps/{type}.conll', f'/content/drive/MyDrive/fillers/{type}.txt', f'/content/drive/MyDrive/generated/{type}.conll', album=(type == 'albums'))
  else:
    generationnames(f'/content/drive/MyDrive/examples/{type}.conll', f'/content/drive/MyDrive/gaps/{type}.conll', f'/content/drive/MyDrive/fillers/{type}.txt', f'/content/drive/MyDrive/generated/{type}.conll')

In [ ]:
with open('generated.conll', 'w') as f:
  for type in types:
    with open(f'generated/{type}.conll', 'r') as f2:
      f.write(f2.read())

In [ ]:
!zip -r gaps.zip gaps/

  adding: gaps/ (stored 0%)
  adding: gaps/districts.conll (stored 0%)
  adding: gaps/cinemas.conll (deflated 58%)
  adding: gaps/artists.conll (deflated 37%)
  adding: gaps/games.conll (deflated 10%)
  adding: gaps/numbers.conll (deflated 22%)
  adding: gaps/tvshows.conll (deflated 25%)
  adding: gaps/names.conll (deflated 12%)
  adding: gaps/books.conll (deflated 20%)
  adding: gaps/movies.conll (deflated 39%)
  adding: gaps/playlists.conll (deflated 42%)
  adding: gaps/days.conll (deflated 29%)
  adding: gaps/albums.conll (stored 0%)
  adding: gaps/cities.conll (deflated 51%)
  adding: gaps/restaurants.conll (deflated 27%)
  adding: gaps/dishes.conll (deflated 22%)
  adding: gaps/months.conll (deflated 39%)
